In [1]:
import os
import sys
sys.path.append("..")

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import utils as utils
import CST as CST
from metrics import recall_m, precision_m, f1_m, auc_m

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

n_classes = len(np.unique(y_train))

x_train = utils.normalize_image(x_train)
x_test = utils.normalize_image(x)

y_train = tf.keras.utils.to_categorical(y_train, num_classes = 10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes = 10)
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 10)


In [3]:
tile_size = 32

alpha = 5
dist_params = {
    "contrast": {"lower": 0.8, "upper": 1.2},
    "color": {"factor": [20,0,20]},
    "blur": {"kernel_size": 1, "sigma": 3.},  # kernel size is 'kernel_size * 2 + 1'
    "brightness": {"max_delta":0.3}
}

model = tf.keras.Sequential([
    tf.keras.applications.ResNet50(include_top=False, input_shape=(32,32,3)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(n_classes, activation='sigmoid')
])


cst = CST.ContrastiveStabilityTraining(
    model=model,
    tile_size=tile_size,
    dist_params=dist_params,
    alpha=alpha
)

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [4]:
opt = tf.keras.optimizers.Adam(lr=1e-3, amsgrad=True)
# opt = tf.keras.optimizers.SGD(lr=1e-4)
metrics = ["categorical_crossentropy", recall_m, precision_m, f1_m, auc_m]
loss = tf.keras.losses.categorical_crossentropy

cst.compile_cst(optimizer=opt, metrics=metrics, loss=loss)

In [ ]:
# train parameters
save_all_epochs = True
model_save_path = "../models"
model_name = "cst_cifar_10"
save_metrics = True
epochs = 100



cst.train_cst(
    x=x_train,
    y=y_train,
    validation_data=(x_test,y_test),
    save_all_epochs=save_all_epochs,
    model_save_path=model_save_path,
    model_name=model_name,
    save_metrics=save_metrics,
    epochs=epochs,
    class_weight=None
)


# model.compile(optimizer=opt, metrics=metrics, loss="categorical_crossentropy")

# model.fit(
#     x=x_train,
#     y=y_train,
#     validation_data=(x_test,y_test),
# #     save_all_epochs=save_all_epochs,
# #     model_save_path=model_save_path,
# #     model_name=model_name,
# #     save_metrics=save_metrics,
#     epochs=epochs,
# #     class_weight=None
# )

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
49984/50000 [============================>.] - ETA: 0s - loss: 0.5024 - acc: 0.8933 - loss_1: 0.5024 - categorical_crossentropy: 0.4113 - recall_m: 0.9275 - precision_m: 0.8311 - f1_m: 0.8649 - auc_m: 0.9585class weights saved to path: 
cst_cifar_101.h5
50000/50000 [==============================] - 192s 4ms/step - loss: 0.5023 - acc: 0.8933 - loss_1: 0.5023 - categorical_crossentropy: 0.4112 - recall_m: 0.9275 - precision_m: 0.8310 - f1_m: 0.8649 - auc_m: 0.9585 - val_loss: 15.7077 - val_acc: 0.1000 - val_loss_1: 15.7077 - val_categorical_crossentropy: 13.0331 - val_recall_m: 0.2000 - val_precision_m: 0.1000 - val_f1_m: 0.1333 - val_auc_m: 0.9353
Epoch 2/100
46688/50000 [===========================>..] - ETA: 11s - loss: 0.2044 - acc: 0.8996 - loss_1: 0.2044 - categorical_crossentropy: 0.1878 - recall_m: 0.9579 - precision_m: 0.8795 - f1_m: 0.9168 - auc_m: 0.9196

#### load cifar-c
TODO: https://stackoverflow.com/questions/4256107/running-bash-commands-in-python

In [ ]:
# mkdir -p ./data/cifar
# curl -O https://zenodo.org/record/2535967/files/CIFAR-10-C.tar
# tar -xvf CIFAR-100-C.tar -C data/cifar/

In [ ]:
cifar_c_path = "../data/CIFAR-10-C"
img_corr = np.load(os.path.join(cifar_c_path, "brightness.npy"))
labels = np.load(os.path.join(cifar_c_path, "labels.npy"))

print(img_corr.shape)